In [1]:
from langchain_community.llms import Ollama

# Ollama

Download Llama2 or Mistral : `ollama run llama2` or `ollama run mistral`
(Keep the terminal open)

Pull the model to your project : `ollama pull llama2` or `ollama pull mistral`

In [2]:
# Load the model, I choose Mistral

ollama = Ollama(model="mistral")

In [3]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="Tu es un cuisinier hors pair, tu as pour but de donner des idées de plat à un utilisateur."),
    HumanMessage(content="Salut, comment vas-tu ? Je ne sais pas quoi manger ce soir. J'ai envie de préparer un plat simple et rapide. tu as des idées ?"),
]

In [4]:
res = ollama.invoke(messages)

In [5]:
print(res)

 Salut! J'ai l'honneur de vous accueillir dans ma cuisine virtuelle. Oui, je peux vous proposer quelques idées simples et rapides pour votre repas ce soir.

1) **Pâte sandwiche au jambon et fromage**: Preparez une pâte feuilletée et faites-en des rondels. Placez sur chaque rondel un morceau de jambon cru, une tranche de fromage de chèvre ou de fromage de chalet, puis enroulez chaque rondel dans la pâte. Cuire les sandwiches dans du beurre fondue à feu doux jusqu'à ce qu'ils soient dorés et servir avec une salade verte.

2) **Omelette à la provencale**: Ciselez une courgette, une aubergine et un poivron rouge. Faites les revenir légèrement dans de l'huile d'olive. Battez des œufs avec du sel, de la poivre et de la romaine. Faites cuire l'omelette dans une poêle à manche haute et terminez-la en four. Servez-la chaude garnie de la provence aux légumes rôlés.

3) **Penne aux tomates cerises**: Faites revenir des tomates cerises avec un morceau d'oignon finement haché et une pincée de poivr

In [6]:
# Append response to messages

messages.append(res)

# Prompt Engineering

In [7]:
prompt = HumanMessage(content="Cite moi 3 plats français très connus.")

messages.append(prompt)

In [8]:
res = ollama.invoke(messages)

In [9]:
print(res)

 Certainement! Voici trois plats français très populaires et appréciés :

1) **Coq au Vin**: C'est un plat de volaille préparé à base de vin rouge, lardons, oignons, champignons, haricots rouges et gros poivrons. Le coq est mariné dans le vin rouge avant d'être cuit lentement jusqu'à ce qu'il soit tendre et savoureux.

2) **Bouillabaisse**: C'est un plat de poissons méditerranéens préparé à base de bouillon de marin et d'épices telles que la tomate, l'aneth, la baie mioule et les ail et oignon hachés. Les poissons utilisés dans ce plat sont traditionnellement les gros poissons de mer comme le congre, le rascasse et le rouget.

3) **Crêpes**: Les crêpes sont un plat populaire en France, qu'on peut remplir de nombreuses façons. Elles peuvent être mangées au sucre ou à la saucisse, avec des fruits de mer, des œufs ou du fromage. Les crêpes sont traditionnellement préparées à l'aide d'une grosse planchette et sont cuites sur une surface plate chaude jusqu'à ce qu'elles soient translucides 

---
## Load documents

In [1]:
from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document

DATA_PATH = "/home/matthieu/UbuntuData/PycharmProjects/FaithBot/Bible Data/All Books"

In [2]:
def load_documents() -> list[Document]:
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    print(f"Loaded {len(documents)} documents")
    print(f"Load Documents, Done!")
    return documents

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


def split_documents(documents: list[Document]):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True
    )

    chunks = splitter.split_documents(documents)
    print(f"Number of chunks: {len(chunks)} for {len(documents)} documents")

    document = chunks[3]
    print(f"Page content : {document.page_content}")
    # print(f"Page number : {document.page_number}")
    print(document.metadata)

    print(type(chunks))
    print(f"Split Documents, Done!")
    return chunks

# Vector Storage : Chroma

In [2]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# Embedding :
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=embedding_model)

In [3]:
import os
import shutil
from langchain.vectorstores.chroma import Chroma

CHROMA_PATH = "chroma"

In [6]:
def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=CHROMA_PATH)
    db.persist()

    print(f"Saved {len(chunks)} documents to Chroma {CHROMA_PATH}")
    print(f"Save to Chroma, Done!")

In [7]:
def generate_database():
    documents = load_documents()
    chunks = split_documents(documents)
    save_to_chroma(chunks)

In [8]:
generate_database()

Loaded 1255 documents
Load Documents, Done!
Number of chunks: 5334 for 1255 documents
Page content : v20

"'Everything that she lies on in her impurity shall be unclean. Everything also that she sits on shall be unclean.

v21

Whoever touches her bed shall wash his clothes, and bathe himself in water, and be unclean until the evening.

v22

Whoever touches anything that she sits on shall wash his clothes, and bathe himself in water, and be unclean until the evening.

v23

If it is on the bed, or on anything she sits on, when he touches it, he shall be unclean until the evening.

v24

"'If any man lies with her, and her monthly flow is on him, he shall be unclean seven days; and every bed he lies on shall be unclean.

v25

"'If a woman has a discharge of her blood many days not in the time of her period, or if she has a discharge beyond the time of her period, all the days of the discharge of her uncleanness shall be as in the days of her period. She is unclean.

v26
{'source': '/home/m

---
## Querying Chroma

In [10]:
import argparse
from dataclasses import dataclass
from langchain_community.llms import Ollama
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""


# Embedding :
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=embedding_model)

In [13]:
def query_chroma(query_text: str):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

    # Search in the database
    results = db.similarity_search_with_relevance_scores(query_text)
    # print(results)
    if len(results) == 0 or results[0][1] < 0.2:
        print("No relevant results found")
        return
    
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    user_prompt = prompt_template.format(context=context_text, question=query_text)
    print(f"Prompt: {user_prompt}")
    
    model = Ollama(model="mistral")
    response_text = model.invoke([HumanMessage(content=user_prompt)])
    
    sources = [doc.metadata["source"] for doc, _score in results]
    formatted_response = f"Response : {response_text}\nSources : {sources}"
    print(formatted_response)
    

In [15]:
query_chroma("How Noah's boat was made ?")

Prompt: Human: 
Answer the question based only on the following context:

v8

Clean animals, unclean animals, birds, and everything that creeps on the ground

v9

went by pairs to Noah into the ship, male and female, as God commanded Noah.

v10

After the seven days, the floodwaters came on the earth.

v11

In the six hundredth year of Noah's life, in the second month, on the seventeenth day of the month, on that day all the fountains of the great deep burst open, and the sky's windows opened.

v12

It rained on the earth forty days and forty nights.

v13

In the same day Noah, and Shem, Ham, and Japheth--the sons of Noah--and Noah's wife and the three wives of his sons with them, entered into the ship--

v14

they, and every animal after its kind, all the livestock after their kind, every creeping thing that creeps on the earth after its kind, and every bird after its kind, every bird of every sort.

v15

Pairs from all flesh with the breath of life in them went into the ship to Noah.